In [ ]:
# Homeostasis Explorer (minimal): System + Disturbance + Roles (no prompts/hints/answers)

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Dropdown, Button, HBox, VBox, Output, HTML

plt.rcParams["figure.figsize"] = (8,4)

SYSTEMS = {
    "Glucose": {
        "setpoint": 90.0,
        "units": "mg/dL",
        "events": {
            "Meal (carbs) – hyperglycemia": {
                "disturbance_magnitude": +40.0,
                "roles": {
                    "Sensor": "Pancreatic β-cells (detect ↑ glucose)",
                    "Control Center": "Pancreatic β-cells (insulin secretion)",
                    "Effector(s)": "Insulin → liver (↑glycogenesis, ↓gluconeogenesis), muscle/adipose (↑GLUT4 uptake)"
                },
                "note": "Hyperglycemia evokes insulin to lower glucose."
            },
            "Light exercise / fasting – hypoglycemia": {
                "disturbance_magnitude": -20.0,
                "roles": {
                    "Sensor": "Pancreatic α-cells (detect ↓ glucose)",
                    "Control Center": "Pancreatic α-cells (glucagon secretion)",
                    "Effector(s)": "Glucagon → liver (↑glycogenolysis, ↑gluconeogenesis); adipose (↑lipolysis)"
                },
                "note": "Hypoglycemia evokes glucagon and counter-regulatory hormones to raise glucose."
            }
        },
        "k_time_constant": 6.0,
        "feedback_gain": 0.35
    },
    
    "Temperature": {
        "setpoint": 37.0,
        "units": "°C",
        "events": {
            "Cold exposure (↓T)": {
                "disturbance_magnitude": -5.0,
                "roles": {
                    "Sensor": "Peripheral & central thermoreceptors (detect ↓temp)",
                    "Control Center": "Hypothalamus (thermoregulatory center)",
                    "Effector(s)": "Shivering, cutaneous vasoconstriction, (± brown fat thermogenesis)"
                },
                "note": "Cold evokes heat production & conservation."
            },
            "Heat exposure (↑T)": {
                "disturbance_magnitude": +2.5,
                "roles": {
                    "Sensor": "Peripheral & central thermoreceptors (detect ↑temp)",
                    "Control Center": "Hypothalamus",
                    "Effector(s)": "Sweating, cutaneous vasodilation"
                },
                "note": "Heat evokes heat dissipation."
            }
        },
        "k_time_constant": 5.0,
        "feedback_gain": 0.40
    },
    
    "Blood Pressure (MAP)": {
        "setpoint": 95.0,
        "units": "mmHg (~MAP)",
        "events": {
            "Standing quickly (orthostatic ↓MAP)": {
                "disturbance_magnitude": -15.0,
                "roles": {
                    "Sensor": "Baroreceptors (carotid sinus, aortic arch) sense ↓ stretch",
                    "Control Center": "Medullary cardiovascular centers",
                    "Effector(s)": "↑HR, ↑contractility, arteriolar vasoconstriction, venoconstriction"
                },
                "note": "Baroreflex counters an acute drop in MAP."
            },
            "Pain/startle (↑MAP surge)": {
                "disturbance_magnitude": +10.0,
                "roles": {
                    "Sensor": "Baroreceptors sense ↑ stretch",
                    "Control Center": "Medullary cardiovascular centers",
                    "Effector(s)": "↑Vagal tone/↓sympathetic tone → ↓HR, ↓contractility, vasodilation"
                },
                "note": "Baroreflex counters an acute rise in MAP."
            }
        },
        "k_time_constant": 4.5,
        "feedback_gain": 0.45
    },
    
    "pH": {
        "setpoint": 7.40,
        "units": "pH",
        "events": {
            "Hypoventilation (resp. acidosis tendency)": {
                "disturbance_magnitude": -0.10,
                "roles": {
                    "Sensor": "Central & peripheral chemoreceptors (↑CO₂/H⁺)",
                    "Control Center": "Medullary respiratory centers",
                    "Effector(s)": "↑Ventilation (↑RR and/or tidal volume) to blow off CO₂"
                },
                "note": "Ventilation rapidly raises pH back toward 7.4."
            },
            "Hyperventilation (resp. alkalosis tendency)": {
                "disturbance_magnitude": +0.10,
                "roles": {
                    "Sensor": "Central & peripheral chemoreceptors (↓CO₂/H⁺)",
                    "Control Center": "Medullary respiratory centers",
                    "Effector(s)": "↓Ventilation (↓RR and/or tidal volume) to retain CO₂"
                },
                "note": "Ventilation lowers pH back toward 7.4."
            }
        },
        "k_time_constant": 3.5,
        "feedback_gain": 0.50
    }
}

def simulate(system_name, event_name, duration=60.0, dt=0.1):
    sys = SYSTEMS[system_name]
    setpoint = sys["setpoint"]
    k = 1.0 / sys["k_time_constant"]
    g = sys["feedback_gain"]
    t = np.arange(0.0, duration + dt, dt)
    x = np.ones_like(t) * setpoint
    D = np.zeros_like(t)
    mag = sys["events"][event_name]["disturbance_magnitude"]
    D[(t >= 5.0) & (t < 15.0)] = mag
    for i in range(1, len(t)):
        error = x[i-1] - setpoint
        feedback = -g * error
        dxdt = D[i-1] - k*error + feedback
        x[i] = x[i-1] + dxdt * dt
    return t, x, setpoint

title = HTML("<h2>Homeostasis Explorer: Negative Feedback</h2>"
             "<p>Select a <b>System</b> and a <b>Disturbance</b>, then click <b>Run Simulation</b>. "
             "Roles update based on the disturbance; the plot shows the variable returning toward its set point.</p>")

w_system = Dropdown(options=list(SYSTEMS.keys()), value="Glucose", description="System:")
w_event  = Dropdown(description="Disturbance:")
btn_run  = Button(description="Run Simulation")

out_plot = Output()
out_info = HTML()

def update_event_options(*args):
    events = list(SYSTEMS[w_system.value]["events"].keys())
    w_event.options = events
    w_event.value = events[0]
update_event_options()
w_system.observe(update_event_options, names="value")

def render():
    out_plot.clear_output(wait=True)
    sys = SYSTEMS[w_system.value]
    evt = sys["events"][w_event.value]
    t, x, setpoint = simulate(w_system.value, w_event.value, duration=60.0, dt=0.1)
    with out_plot:
        plt.figure()
        plt.plot(t, x, label="Observed variable")
        plt.axhline(setpoint, linestyle="--", label=f"Set point = {setpoint} {sys['units']}")
        plt.axvspan(5.0, 15.0, alpha=0.1, label="Disturbance window")
        plt.xlabel("Time (a.u.)")
        plt.ylabel(f"{w_system.value} ({sys['units']})")
        plt.title(f"{w_system.value} response to: {w_event.value}")
        plt.legend(loc="best")
        plt.show()
    roles = evt["roles"]
    out_info.value = (
        f"<h3>Feedback Roles — <i>{w_event.value}</i></h3>"
        f"<ul>"
        f"<li><b>Sensor:</b> {roles['Sensor']}</li>"
        f"<li><b>Control Center:</b> {roles['Control Center']}</li>"
        f"<li><b>Effector(s):</b> {roles['Effector(s)']}</li>"
        f"</ul>"
        f"<p><i>Note:</i> {evt['note']}</p>"
    )

btn_run.on_click(lambda _: render())
controls = HBox([w_system, w_event, btn_run])
ui = VBox([title, controls, out_plot, out_info])
display(ui)
render()
